In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/'My Drive'/spidermen

/content/gdrive/My Drive/spidermen


In [0]:
import numpy as np
import torch.utils.data
from torch.nn import functional as F
import cv2
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.functional import pad
from torch.nn.modules import Module
from torch.nn.modules.utils import _single, _pair, _triple
from __future__ import print_function
import argparse
import os
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
import torch.backends.cudnn as cudnn

### Source: The function below is the  umeyama function from scikit-image library
### specifically the scikit-image/skimage/transform/_geometric.py file
###  and is used to warp the inputted image while 
### training the autoencoder and decoder

def umeyama(src, dst, estimate_scale):
    """Estimate N-D similarity transformation with or without scaling.
    Parameters
    ----------
    src : (M, N) array
        Source coordinates.
    dst : (M, N) array
        Destination coordinates.
    estimate_scale : bool
        Whether to estimate scaling factor.
    Returns
    -------
    T : (N + 1, N + 1)
        The homogeneous similarity transformation matrix. The matrix contains
        NaN values only if the problem is not well-conditioned.
    References
    ----------
    .. [1] "Least-squares estimation of transformation parameters between two
            point patterns", Shinji Umeyama, PAMI 1991, DOI: 10.1109/34.88573
    """

    num = src.shape[0]
    dim = src.shape[1]

    # Compute mean of src and dst.
    src_mean = src.mean(axis=0)
    dst_mean = dst.mean(axis=0)

    # Subtract mean from src and dst.
    src_demean = src - src_mean
    dst_demean = dst - dst_mean

    # Eq. (38).
    A = np.dot(dst_demean.T, src_demean) / num

    # Eq. (39).
    d = np.ones((dim,), dtype=np.double)
    if np.linalg.det(A) < 0:
        d[dim - 1] = -1

    T = np.eye(dim + 1, dtype=np.double)

    U, S, V = np.linalg.svd(A)

    # Eq. (40) and (43).
    rank = np.linalg.matrix_rank(A)
    if rank == 0:
        return np.nan * T
    elif rank == dim - 1:
        if np.linalg.det(U) * np.linalg.det(V) > 0:
            T[:dim, :dim] = np.dot(U, V)
        else:
            s = d[dim - 1]
            d[dim - 1] = -1
            T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V))
            d[dim - 1] = s
    else:
        T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V.T))

    if estimate_scale:
        # Eq. (41) and (42).
        scale = 1.0 / src_demean.var(axis=0).sum() * np.dot(S, d)
    else:
        scale = 1.0

    T[:dim, dim] = dst_mean - scale * np.dot(T[:dim, :dim], src_mean.T)
    T[:dim, :dim] *= scale

    return T

In [0]:
random_transform_args = {
    'rotation_range': 10,
    'zoom_range': 0.05,
    'shift_range': 0.05,
    'random_flip': 0.4,
}


### Uses set values to rotate, zoom and shift the images within the inputted 
### array with a chance of flipping the image. 
### This is used on the image of the OG dataset while creating the the original
### dataset https://github.com/Oldpan/Faceswap-Deepfake-Pytorch

def random_transform(image, rotation_range, zoom_range, shift_range, random_flip):
    h, w = image.shape[0:2]
    rotation = np.random.uniform(-rotation_range, rotation_range)
    scale = np.random.uniform(1 - zoom_range, 1 + zoom_range)
    transformationOnX = np.random.uniform(-shift_range, shift_range) * w
    ty = np.random.uniform(-shift_range, shift_range) * h
    mat = cv2.getRotationMatrix2D((w // 2, h // 2), rotation, scale)
    mat[:, 2] += (transformationOnX, ty)
    result = cv2.warpAffine(image, mat, (w, h), borderMode=cv2.BORDER_REPLICATE)
    #randomly flips the image
    if np.random.random() < random_flip:
        result = result[:, ::-1]
    return result


# This function is used to generate a pair of random warped images from the 
# face image which is used to train the autoencoder and decoders. This 
# was sourced from https://github.com/Oldpan/Faceswap-Deepfake-Pytorch

def random_warp(image):
    assert image.shape == (256, 256, 3)
    range_ = np.linspace(128 - 80, 128 + 80, 5)
    mapx = np.broadcast_to(range_, (5, 5))
    mapy = mapx.T

    mapx = mapx + np.random.normal(size=(5, 5), scale=5)
    mapy = mapy + np.random.normal(size=(5, 5), scale=5)

    interp_mapx = cv2.resize(mapx, (80, 80))[8:72, 8:72].astype('float32')
    interp_mapy = cv2.resize(mapy, (80, 80))[8:72, 8:72].astype('float32')

    warped_image = cv2.remap(image, interp_mapx, interp_mapy, cv2.INTER_LINEAR)

    src_points = np.stack([mapx.ravel(), mapy.ravel()], axis=-1)
    dst_points = np.mgrid[0:65:16, 0:65:16].T.reshape(-1, 2)
    mat = umeyama(src_points, dst_points, True)[0:2]

    target_image = cv2.warpAffine(image, mat, (64, 64))

    return warped_image, target_image

In [0]:
## creates training batches to be used in the training loop of the autoencoders
## Uses Oldpan's random_warp and random_transform functions to
## warp and transform images prior to inputting into autoencoder.

## InputType: List[images], int
"""
Input Description:
images: These would be the list of images loaded into memory to be 
used for training

batch_size: The batch size for each step of training
"""
## Output: List[images], List[images]
def generateTrainingData(images, batch_size):
    ### selects a random batch_size number of images from the dataset
    ix = np.random.randint(len(images), size=batch_size)
    
    for i in range(len(ix)):
        img = images[ix[i]]
        img = random_transform(img, **random_transform_args)
        warpedImage, TargetImage = random_warp(img)
        
        if i == 0:
            warpedImages = np.empty((batch_size,) + warpedImage.shape, warpedImage.dtype)
            targetImages = np.empty((batch_size,) + TargetImage.shape, warpedImage.dtype)

        warpedImages[i] = warpedImage
        targetImages[i] = TargetImage

    return warpedImages, targetImages

In [0]:
### This is PyTorch source code for the ConvNd module which is being
### redefined as the Conv2d forward method is overriden

class _ConvNd(Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride,
                 padding, dilation, transposed, output_padding, groups, bias):
        super(_ConvNd, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        if transposed:
            self.weight = Parameter(torch.Tensor(
                in_channels, out_channels // groups, *kernel_size))
        else:
            self.weight = Parameter(torch.Tensor(
                out_channels, in_channels // groups, *kernel_size))
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

### PyTorch source code, with the forward method being overridden to deal with 
### padding = 'same' option
class Conv2d(_ConvNd):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)
        super(Conv2d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            False, _pair(0), groups, bias)

    def forward(self, input):
        return conv2d_same_padding(input, self.weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)


### A custom  conv2d method written as Torch doesn't have "padding='same'" option
### as seen in TensorFlow Keras' Conv2D layers. The code below has been hacked together from 
### TensorFlow Keras source code 
### and sourced from https://github.com/hanqingguo/deepfake-pytorch
### The override is created so that the output is the same length as the input
### mimicking the "padding = same" option in TensorFlow Keras.
def conv2d_same_padding(input, weight, bias=None, stride=1, padding=1, dilation=1, groups=1):

    input_rows = input.size(2)
    filter_rows = weight.size(2)
    effective_filter_size_rows = (filter_rows - 1) * dilation[0] + 1
    out_rows = (input_rows + stride[0] - 1) // stride[0]
    padding_needed = max(0, (out_rows - 1) * stride[0] + effective_filter_size_rows -
                  input_rows)
    padding_rows = max(0, (out_rows - 1) * stride[0] +
                        (filter_rows - 1) * dilation[0] + 1 - input_rows)
    rows_odd = (padding_rows % 2 != 0)
    padding_cols = max(0, (out_rows - 1) * stride[0] +
                        (filter_rows - 1) * dilation[0] + 1 - input_rows)
    cols_odd = (padding_rows % 2 != 0)

    if rows_odd or cols_odd:
        input = pad(input, [0, int(cols_odd), 0, int(rows_odd)])

    return F.conv2d(input, weight, bias, stride,
                  padding=(padding_rows // 2, padding_cols // 2),
                  dilation=dilation, groups=groups)

In [0]:
def get_image_paths(dir):
    return [x.path for x in os.scandir(dir) if x.name.endswith(".jpg") or x.name.endswith(".png")
            or x.name.endswith(".JPG")]

def normalize_images(imageList):
  imageList = imageList / 255.0
  return imageList

def load_images(img_path):
    imageList = (cv2.resize(cv2.imread(fn), (256, 256)) for fn in img_path)

    for i, image in enumerate(imageList):
        if i == 0:
            finalImageList = np.empty((len(img_path),) + image.shape, dtype=image.dtype)
        finalImageList[i] = image
    return finalImageList

def get_transpose_axes(n):
    if n % 2 == 0:
        y_axes = list(range(1, n - 1, 2))
        x_axes = list(range(0, n - 1, 2))
    else:
        y_axes = list(range(0, n - 1, 2))
        x_axes = list(range(1, n - 1, 2))
    return y_axes, x_axes, [n - 1]


def stack_images(images):
    images_shape = np.array(images.shape)
    new_axes = get_transpose_axes(len(images_shape))
    new_shape = [np.prod(images_shape[x]) for x in new_axes]
    return np.transpose(
        images,
        axes=np.concatenate(new_axes)
    ).reshape(new_shape)

In [0]:
import torch
import torch.utils.data
from torch import nn, optim

def toTensor(img):
    img = torch.from_numpy(img.transpose((0, 3, 1, 2)))
    return img


def var_to_np(img_var):
    return img_var.data.cpu().numpy()

### A simple custom layer within the NN which flattens the tensor being 
### passed through it
### sourced from https://github.com/hanqingguo/deepfake-pytorch
class Flatten(nn.Module):

    def forward(self, input):
        output = input.view(input.size(0), -1)
        return output

### A simple custom layer within the NN which reshapes the tensor being 
### passed through it
### sourced from https://github.com/hanqingguo/deepfake-pytorch
class Reshape(nn.Module):

    def forward(self, input):
        output = input.view(-1, 1024, 4, 4)  # channel * 4 * 4

        return output
      
# https://gist.github.com/t-ae/6e1016cc188104d123676ccef3264981
# need to read up on pixelshufflers
class _PixelShuffler(nn.Module):
    def forward(self, input):
        batch_size, c, h, w = input.size()
        rh, rw = (2, 2)
        oh, ow = h * rh, w * rw
        oc = c // (rh * rw)
        out = input.view(batch_size, rh, rw, oc, h, w)
        out = out.permute(0, 3, 4, 1, 5, 2).contiguous()
        out = out.view(batch_size, oc, oh, ow)  # channel first

        return out


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        """
        self.encoder = nn.Sequential(
            _ConvLayer(3, 128),
            _ConvLayer(128, 256),
            _ConvLayer(256, 512),
            _ConvLayer(512, 1024),
            Flatten(),
            nn.Linear(1024 * 4 * 4, 1024),
            nn.Linear(1024, 1024 * 4 * 4),
            Reshape(),
            _UpScale(1024, 512),
        )
        """
        
        self.encoder = nn.Sequential(
            Conv2d(3, 128,kernel_size=5, stride=2),
            nn.LeakyReLU(0.1, inplace=True),
            Conv2d(128, 256,kernel_size=5, stride=2),
            nn.LeakyReLU(0.1, inplace=True),
            Conv2d(256, 512,kernel_size=5, stride=2),
            nn.LeakyReLU(0.1, inplace=True),
            Conv2d(512, 1024,kernel_size=5, stride=2),
            nn.LeakyReLU(0.1, inplace=True),
            Flatten(),
            nn.Linear(1024 * 4 * 4, 1024),
            nn.Linear(1024, 1024 * 4 * 4),
            Reshape(),
            Conv2d(1024, 512 * 4,kernel_size=3),
            nn.LeakyReLU(0.1, inplace=True),
            _PixelShuffler())

        self.decoder_A = nn.Sequential(
            Conv2d(512, 256 * 4,kernel_size=3),
            nn.LeakyReLU(0.1, inplace=True),
            _PixelShuffler(),
            Conv2d(256, 128 * 4,kernel_size=3),
            nn.LeakyReLU(0.1, inplace=True),
            _PixelShuffler(),
            Conv2d(128, 64 * 4,kernel_size=3),
            nn.LeakyReLU(0.1, inplace=True),
            _PixelShuffler(),
            Conv2d(64, 3, kernel_size=5, padding=1),
            nn.Sigmoid(),
        )

        self.decoder_B = nn.Sequential(
            Conv2d(512, 256 * 4,kernel_size=3),
            nn.LeakyReLU(0.1, inplace=True),
            _PixelShuffler(),
            Conv2d(256, 128 * 4,kernel_size=3),
            nn.LeakyReLU(0.1, inplace=True),
            _PixelShuffler(),
            Conv2d(128, 64 * 4,kernel_size=3),
            nn.LeakyReLU(0.1, inplace=True),
            _PixelShuffler(),
            Conv2d(64, 3, kernel_size=5, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x, select='A'):
        if select == 'A':
            x = self.encoder(x)
            x = self.decoder_A(x)
        else:
            x = self.encoder(x)
            x = self.decoder_B(x)
        return x

In [0]:
if torch.cuda.is_available():
  print('The GPU is being used to train the model')
  device = torch.device('cuda:0')
  cudnn.benchmark = True
else:
    print('The CPU is being used to train the model. This will be slower.')

The GPU is being used to train the model


In [0]:
print('The datasets are being loaded into RAM... \n Note: This might take some time...')

images_Tobey = get_image_paths("./fiveclips.mp4_faces")
images_Holland = get_image_paths("./holland1.mp4_faces")
images_Tobey = normalize_images(load_images(images_Tobey))
images_Holland = normalize_images(load_images(images_Holland))

The datasets are being loaded into RAM... 
 Note: This might take some time...


In [0]:
model = Autoencoder().to(device)

criterion = nn.L1Loss()
optimizer_1 = optim.Adam([{'params': model.encoder.parameters()},
                          {'params': model.decoder_A.parameters()}]
                         , lr=5e-5, betas=(0.5, 0.999))
optimizer_2 = optim.Adam([{'params': model.encoder.parameters()},
                          {'params': model.decoder_B.parameters()}]
                         , lr=5e-5, betas=(0.5, 0.999))
batchSize = 64
endPoint = 100001
whenToSave = 100


### we check whether a new model is being trained or whether 
### we are continuing to train the same model 
if os.path.isdir('checkpoint'):
    try:
        checkpoint = torch.load('./checkpoint/autoencoder.t7')
        model.load_state_dict(checkpoint['state'])
        startPoint = checkpoint['epoch']
    except FileNotFoundError:
        pass
else:
    startPoint = 0

count = 0
for epoch in range(start_epoch, end_epoch):
  loopBatchSize = batchSize
  
  if count == 1:
    break
  
  
  warped_B, target_B = generateTrainingData(images_Holland, loopBatchSize)
  warped_A, target_A = generateTrainingData(images_Tobey, loopBatchSize)
  warped_B = toTensor(warped_B)
  warped_A = toTensor(warped_A)
  target_B = toTensor(target_B)
  target_A = toTensor(target_A)
  
  warped_A = warped_A.to(device).float()
  target_A = target_A.to(device).float()
  warped_B = warped_B.to(device).float()
  target_B = target_B.to(device).float()
  
  optimizer_1.zero_grad()
  optimizer_2.zero_grad()
    
  warped_A = model(warped_A, 'A')
  warped_B = model(warped_B, 'B')
        
  loss1 = criterion(warped_A, target_A)
  loss2 = criterion(warped_B, target_B)
  loss = loss1.item() + loss2.item()
  loss1.backward()
  loss2.backward()
  optimizer_1.step()
  optimizer_2.step()
  
  print('epoch: {}, lossTobey:{}, lossHolland:{}'.format(epoch, loss1.item(), loss2.item()))
  
  count += 1

  if epoch % whenToSave == 0:
    print('===> Saving models...')
    state = {
        'state': model.state_dict(),'epoch': epoch}
    if not os.path.isdir('checkpoint'):
      os.mkdir('checkpoint')
    torch.save(state, './checkpoint/autoencoder.t7')

RuntimeError: ignored

/content/gdrive/My Drive/spidermen


In [0]:
print('===> Saving final model')
state = {'state': model.state_dict(),'epoch': epoch}
if not os.path.isdir('checkpoint'):
  os.mkdir('checkpoint')
torch.save(state, './checkpoint/autoencoder.t7')

===> Saving final model


In [0]:
import torch

In [0]:
if torch.cuda.is_available():
  device = torch.device("cuda")

In [0]:
device


device(type='cuda')

In [0]:
y = torch.torch.Tensor

TypeError: ignored